# MIT Stars & Planets Workshop 2025

## Stellar contamination with `fleck` – Brett Morris


##### Contents

* <a href='#background'>Background</a>


##### Links

* [workshop details](https://seec.gsfc.nasa.gov/News_and_Events/Spectral_Retrieval_Tutorial_2024.html)
* `fleck`: [source](https://github.com/bmorris3/fleck), [docs](https://fleck.readthedocs.io/en/latest/fleck/jax.html), [JOSS paper](https://joss.theoj.org/papers/10.21105/joss.02103), [ApJ paper](https://ui.adsabs.harvard.edu/abs/2020ApJ...893...67M/abstract)
* `shone`: [source](https://github.com/bmorris3/shone/), [docs](https://shone.readthedocs.io/en/latest/)
* `jax`: [source](https://github.com/google/jax), [docs](https://jax.readthedocs.io/en/latest/)

<br /><br />

<a id='physical_model'></a>

### Background

#### Atmospheres from transit depths

During an exoplanet transit, we usually imagine that the change in flux $\Delta F$ relative to the mean flux $\bar{F}$,
\begin{equation}
\frac{\Delta F}{F} \propto \left(\frac{R_p}{R_\star}\right)^2,
\end{equation}
where $R_p$ and $R_\star$ are the planetary and stellar radii. All spectral flux densities in this notebook $F$ are functions of wavelength (implicitly $F_\lambda$). The proportionality becomes an equality without stellar limb darkening. 

We rely on this proportionality to detect exoplanet atmospheres by measuring the spectroscopic variations in the transit depth. The apparent radius of a planet with an atmosphere varies with wavelength due to atomic and molecular opacity, $R_p(\lambda)$. We match the variation in transit depth with wavelength to the sources of opacity in the planetary atmosphere,
\begin{equation}
\frac{\Delta F_\lambda}{\bar{F}} \propto \left(\frac{R_p(\lambda)}{R_\star}\right)^2.
\end{equation}

<br /><br />

#### Stellar contamination

The equations above need modification if the stellar surface is inhomogeneous. A star may have cool regions (e.g. spots) and/or hot regions (e.g. faculae) due to magnetic activity. Let the stellar surface have spot covering fraction $f_{\rm spot}$ which emits spectrum $F_{\rm spot}$, while the remainder is the photosphere which emits $F_{\rm phot}$, then the full stellar hemisphere has the spectrum:
\begin{equation}
F_{\rm hem} = (1 - f_{\rm spot}) F_{\rm phot} + f_{\rm spot} F_{\rm spot}.
\end{equation}
The transit depth in the presence of unocculted active regions is
\begin{eqnarray}
\frac{\Delta F}{F_{\rm hem}} &\propto& \left(\frac{R_p}{R_\star}\right)^2 \frac{F_{\rm phot}}{F_{\rm hem}}.
\end{eqnarray}
If the active star $F_{\rm hem}$ is brighter than the photosphere $F_{\rm phot}$ at a given wavelength, $F_{\rm phot}/F_{\rm hem} < 1$, and the transit is shallower than the same star without activity. Unocculted cool regions make transits deeper. 
<br /><br />

#### Implications

The ratio of the photosphere-only spectrum and photosphere-plus-active-region spectrum, $F_{\rm phot}/F_{\rm hem}$, is a spectral quantity. If the temperature difference between the photosphere and the active regions is large, the major sources of atomic and molecular opacity may change, producing significantly different emission spectra. In the solar photosphere ($T_{\rm eff} = 5770$ K), the center of a sunspot can be several thousand degrees cooler than the photosphere. That's cool enough to detect stable [water molecules in the solar photosphere](https://www.science.org/doi/10.1126/science.7761830).

<div class="alert alert-block alert-info">
<b>Brett's nightmare:</b> a distant observer of a small transiting an active star would have to disentangle spectroscopic water features from the atmospheres of the planet and star.
</div>

### Spectrophotometry of active stars with `fleck`

`fleck` is a Python/JAX package for modeling active stars with (or without) transiting planets (papers: [ApJ](https://ui.adsabs.harvard.edu/abs/2020ApJ...893...67M/abstract), [JOSS](https://joss.theoj.org/papers/10.21105/joss.02103)). `fleck` computes the observed spectrum throughout stellar rotation and during transits while accounting for circular active regions with distinct emission spectra from the quiescent photosphere.

---

### If you're using Google Colab:

In [ ]:
%%writefile requirements.txt

matplotlib
astropy
ipywidgets
expecto
git+https://github.com/bmorris3/shone@condensation
git+https://github.com/bmorris3/fleck
jupyterlab

In [ ]:
%%bash

pip install -U -r requirements.txt

In Google Colab, restart the notebook before you run the next cell.



---

In [ ]:
import os
os.environ['JAX_PLATFORMS'] = 'cpu' # needed for Google Colab
from jax import config
config.update('jax_enable_x64', True)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from IPython.display import display
from ipywidgets import interactive, VBox, HBox, FloatSlider

from jax import numpy as jnp

import astropy.units as u
from astropy.constants import m_p

from shone.opacity import Opacity
from shone.transmission import heng_kitzmann_2017

from expecto import get_spectrum
from fleck.jax import ActiveStar, bin_spectrum

Download PHOENIX model spectra for an M4V star, and cooler and warmer active regions.

In [ ]:
wavelength = np.geomspace(0.6, 5, 250)  # [µm]
times = np.linspace(-0.04, 2.04, 100)  # time for each spectrum

# Set model spectrum binning preferences:
wl_bins = np.concatenate([wavelength, [wavelength[-1] + (wavelength[-1] - wavelength[-2])]]) * u.um
kwargs = dict(
    bins=wl_bins,
    min=wl_bins.min(),
    max=wl_bins.max(),
    log=False
)

# Download and bin PHOENIX model spectra:
phot, cool, hot = [
    bin_spectrum(
        get_spectrum(T_eff=T_eff, log_g=5.0, cache=True), **kwargs
    )
    for T_eff in [3000, 2300, 3200]
]

Construct an `ActiveStar` instance for this star, and one warm and one cool active region:

In [ ]:
active_star = ActiveStar(
    times=times,
    inclination=np.pi/2,  # stellar inc [rad]
    T_eff=phot.meta['PHXTEFF'],
    wavelength=phot.wavelength.to_value(u.m),
    phot=phot.flux.value,
    P_rot=2.0  # [days]
)

spot_cool = dict(
    lon=-1.2,  # [rad]
    lat=1.3,  # [rad]
    rad=0.04,  # [R_star]
    spectrum=cool.flux.value,
    temperature=cool.meta['PHXTEFF']
)

spot_hot = dict(
    lon=0.2,
    lat=2.2,
    rad=0.02,
    spectrum=hot.flux.value,
    temperature=hot.meta['PHXTEFF']
)

for spot in [spot_cool, spot_hot]:
    active_star.add_spot(**spot)

Show the light curves at each wavelength throughout a rotation, and show the relative spectroscopic variations

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize, to_hex


spectra = [phot, cool, hot]

log_temps = np.log10([spectrum.meta['PHXTEFF'] for spectrum in spectra])

def temp_cmap(x):
    return to_hex(
        plt.cm.YlOrRd_r(
            (np.log10(x) - min(log_temps)) /
            (max(log_temps) - min(log_temps)) * 0.6 + 0.4
        )
    )

rotation = active_star.rotation_spectrum()

fig, ax = plt.subplots(2, 2, figsize=(10, 8), dpi=100)

active_star.plot_star(0, 10, 10, np.pi/2, ax=ax[0, 0])

for spectrum, label in zip(spectra, 'Phot Cool Hot'.split()):
    wavelength_range = (
        (spectrum.wavelength.to_value(u.um) > wavelength.min()) & 
        (spectrum.wavelength.to_value(u.um) < wavelength.max())
    )
    plot_args = (
        spectrum.wavelength[wavelength_range].to_value(u.um), 
        spectrum.flux[wavelength_range], 
    )
    zorder = 100 if label=='Phot' else -1
    ax[0, 1].plot(*plot_args, color='gray', lw=3.5, zorder=zorder)
    ax[0, 1].plot(*plot_args, color=temp_cmap(spectrum.meta['PHXTEFF']), label=label, zorder=zorder)
ax[0, 1].legend()

for lc, wl in zip(rotation.T, wavelength):
    ax[1, 0].plot(times, lc / lc.mean(), color=plt.cm.Spectral_r((wl - wavelength.min()) / np.ptp(wavelength)), alpha=0.5);

mean_spectrum = rotation.mean(0)

for spectrum, time in zip(rotation, times):
    ax[1, 1].plot(wavelength, 1e3 * (spectrum / mean_spectrum - 1), color=plt.cm.viridis((time - times.min()) / np.ptp(times)), alpha=0.5);

mappable = ScalarMappable(Normalize(vmin=wavelength.min(), vmax=wavelength.max()), cmap=plt.cm.Spectral_r)
plt.colorbar(mappable, ax=ax[1, 0], label='')

mappable = ScalarMappable(Normalize(vmin=times.min(), vmax=times.max()), cmap=plt.cm.viridis)
plt.colorbar(mappable, ax=ax[1, 1], label='Time [d]')

unit_string = phot.flux.unit.to_string("latex")

ax[0, 1].set(
    xlabel='Time [d]',
    ylabel=f'$F_\lambda$ [{unit_string}]'
)

ax[1, 0].set(
    xlabel='Time [d]',
    ylabel='Relative Flux'
)

ax[1, 1].set(
    xlabel='Wavelength [µm]',
    ylabel='Relative Flux [ppt]'
)

fig.tight_layout()
plt.show()

Set planet properties:

In [ ]:
R_0 = 1 * u.R_earth  # reference radius
Rstar = 0.2 * u.R_sun  # stellar radius
P_0 = 0.1 * u.bar  # reference pressure
T_0 = 400 * u.K  # reference temperature
kappa_cloud = 1e-2  # cloud opacity [cm2 / g]


# transit parameters for the planet:
planet_parameters = dict(
    inclination = np.pi/2,
    a = 10,
    rp = float(R_0 / Rstar),
    period = 1.5,
    t0 = 0,
    ecc = 0,
    u1 = 0.1,
    u2 = 0.1
)

In [ ]:
# get opacity at several temperatures, all at 1 bar:
temperature = jnp.array([T_0.value])  # [K]
pressure = np.array([0.1]) # [bar]

# in this example we'll use the demo opacities,
# which you *should not use* in real work:
opacity = Opacity.load_demo_species("H2O")
interp_opacity = opacity.get_interpolator()

In [ ]:
def transmission_spectrum(vmr, mmw, g_cgs):
    """
    Compute a transmission spectrum for an atmosphere 
    using the isothermal/isobaric approximation 
    from Heng & Kitzmann (2017).
    """
    # mmw = 28  # mean molecular weight (AMU)
    # vmr = 0.01  # volume mixing ratio
    weight = 44  # weight for CO2
    
    total_opacity = (
        vmr * weight / mmw * 
        interp_opacity(wavelength, temperature, pressure)
    )
    
    # compute the planetary radius as a function of wavelength:
    Rp = heng_kitzmann_2017.transmission_radius_isothermal_isobaric(
        total_opacity + kappa_cloud,
        R_0.cgs.value, P_0, T_0, mmw, g_cgs
    )
    
    # convert to transit depth:
    transit_depth_ppm = 1e6 * (Rp / Rstar.cgs.value) ** 2
    return Rp / Rstar.cgs.value, transit_depth_ppm

Compute and plot a transmission spectrum of this demo atmosphere:

In [ ]:
Rp_Rstar, transit_depth_ppm = transmission_spectrum(vmr=0.01, mmw=28, g_cgs=9.8)

ax = plt.gca()
ax.plot(wavelength, transit_depth_ppm, label=f"{T_0:.0f}")

# add labels for CO2 and H2O features:
label_height = transit_depth_ppm.max() + 500
water_peaks = [1.4, 1.9, 2.7]
for peak in water_peaks:
    ax.annotate("H$_2$O", (peak, label_height), ha='center')

ax.legend(loc=(1.05, 0))
ax.set(
    xlabel='Wavelength [µm]',
    ylabel='Transit depth [ppm]',
    ylim=[None, transit_depth_ppm.max() + 2000]
)

plt.show()

Now here's a big function to produce an interactive plot:

In [ ]:
def plot_transit_contamination(
    active_star, planet_params,
    norm_oot_per_wavelength=True,
    norm_stellar_spectrum=True
):
    active_star.times = np.linspace(-0.04, 0.04, 100)  # transit times

    lc, contam, X, Y, spectrum_at_transit = active_star.transit_model(**planet_params)
    fig = plt.figure(figsize=(10, 5), dpi=100)
    gs = GridSpec(2, 2, figure=fig)

    ax = [
        fig.add_subplot(gs[0, 0]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[:, 1:3]),
    ]

    skip = (len(active_star.wavelength) - 1) // 10

    cmap = lambda i: plt.cm.Spectral_r(
        (active_star.wavelength[i] - active_star.wavelength.min()) /
        active_star.wavelength.ptp()
    )

    if norm_stellar_spectrum:
        scale_relative_to_flux_at_wavelength = 1
    else:
        scale_relative_to_flux_at_wavelength = (
            spectrum_at_transit / spectrum_at_transit.mean()
        )[::skip]

    for i, lc_i in enumerate(
        (lc * scale_relative_to_flux_at_wavelength)[:, ::skip].T
    ):

        if norm_oot_per_wavelength:
            lc_i /= lc_i.mean()

        ax[0].plot(active_star.times, lc_i, color=cmap(skip * i))


    ax[0].set(
        xlabel='Time [d]',
        ylabel='$\\left(F(t)/\\bar{F}\\right)_{\\lambda}$',
    )

    contaminated_depth = 1e6 * contam

    ax[1].plot(
        active_star.wavelength * 1e6,
        contaminated_depth,
        zorder=-3, lw=2.5, color='silver'
    )
    ax[1].scatter(
        active_star.wavelength[::skip] * 1e6, contaminated_depth[::skip].T,
        c=cmap(skip * np.arange(len(active_star.wavelength) // skip + 1)),
        s=50, edgecolor='k', zorder=4
    )

    active_star.plot_star(
        t0=planet_params['t0'],
        rp=planet_params['rp'],
        a=planet_params['a'],
        ecc=planet_params['ecc'],
        inclination=planet_params['inclination'],
        ax=ax[2]
    )

    for sp in ['right', 'top']:
        for axis in ax:
            axis.spines[sp].set_visible(False)

    xticks = np.arange(1, 6)
    ax[1].set(
        xlabel='Wavelength [µm]',
        ylabel='Transit depth [ppm]',
        xscale='log',
        xlim=[
            1e6 * 0.9 * active_star.wavelength.min(),
            1e6 * 1.1 * active_star.wavelength.max()
        ],
        xticks=xticks,
        xticklabels=xticks,
    )
    
    fig.tight_layout()

This function produces the interactive widget:

In [ ]:
def lc_interact(
    lon1, lon2, lat1, lat2, rad1, rad2, log_vmr, mmw, g_cgs
):

    planet_parameters['rp'] = transmission_spectrum(vmr=10 ** log_vmr, mmw=mmw, g_cgs=g_cgs)[0]
    norm_oot_per_wavelength=True
    norm_stellar_spectrum=True
    active_star.lon = jnp.array([lon1, lon2])
    active_star.lat = jnp.array([lat1, lat2])
    active_star.rad = jnp.array([rad1, rad2])

    lc, contam, X, Y, spectrum_at_transit = active_star.transit_model(
        **planet_parameters
    )

    plot_transit_contamination(
        active_star, planet_parameters
    )
    plt.show()

And now let's launch the widget:

In [ ]:
widget = interactive(
    lc_interact,
    lon1=FloatSlider(value=spot_cool['lon'], min=-np.pi, max=np.pi, step=0.001),
    lon2=FloatSlider(value=spot_hot['lon'], min=-np.pi, max=np.pi, step=0.001),
    lat1=FloatSlider(value=spot_cool['lat'], min=0, max=np.pi, step=0.001),
    lat2=FloatSlider(value=spot_hot['lat'], min=0, max=np.pi, step=0.001),
    rad1=FloatSlider(value=spot_cool['rad'], min=0, max=0.35, step=0.001),
    rad2=FloatSlider(value=spot_hot['rad'], min=0, max=0.35, step=0.001),
    log_vmr=FloatSlider(value=-6, min=-8, max=-2, step=0.1),
    mmw=FloatSlider(value=25, min=2, max=50, step=1),
    g_cgs=FloatSlider(value=980, min=500, max=1500, step=10),
);

controls = VBox(widget.children[:-1])
output = widget.children[-1]
display(HBox([output, controls]))

The Earth's mean molecular weight $\textrm{mmw} \sim 28$ AMU. 

The water volume mixing ratio in Earth's stratosphere is roughly $\log(\textrm{VMR}) \sim 10^{-6}$.

Sunspots usually have radii $R_{\rm spot} / R_{\star} \sim$ 0.03.

Typical solar faculae cover 4x more area than spots.